In [2]:
import qutip as qt
import numpy as np

In [195]:
sx = qt.operators.sigmax()
sx

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 1.]
 [1. 0.]]

In [196]:
def hamiltonian_ising(N,omega,delta,V):
  sx = 0.5*qt.operators.sigmax()
  sy = 0.5*qt.operators.sigmay()
  sz = 0.5*qt.operators.sigmaz()
  I = qt.operators.identity(2)

  ising_ham = qt.tensor([I*0]*N)
  for i in range(N):
    hz = qt.tensor([sz if j==i else I for j in range(N)])
    hx = qt.tensor([sx if j==i else I for j in range(N)])
    ising_ham += -1*delta*hz+omega*hx
    for j in range(i + 1, N):
        # Interaction term between qubit i and qubit j
        #print(i,j)
        hzz = qt.tensor([sz if k == i or k == j else I for k in range(N)]) # assume obc
        ising_ham += (V/N)*hzz
  
  #print(hz,hx,hzz)
  
  return(ising_ham)

In [197]:
omega = 1
N = 2
delta = 1
V = 3
hamiltonian=hamiltonian_ising(N,omega,delta,V)
hamiltonian

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-0.625  0.5    0.5    0.   ]
 [ 0.5   -0.375  0.     0.5  ]
 [ 0.5    0.    -0.375  0.5  ]
 [ 0.     0.5    0.5    1.375]]

In [198]:
# validation
sx = 0.5*qt.operators.sigmax()
sy = 0.5*qt.operators.sigmay()
sz = 0.5*qt.operators.sigmaz()
I = qt.operators.identity(2)

ham_t = (V/N)*qt.tensor(sz,sz)+delta*qt.tensor(sx,I)+delta*qt.tensor(I,sx)-omega*qt.tensor(sz,I)-omega*qt.tensor(I,sz)
ham_t

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-0.625  0.5    0.5    0.   ]
 [ 0.5   -0.375  0.     0.5  ]
 [ 0.5    0.    -0.375  0.5  ]
 [ 0.     0.5    0.5    1.375]]

In [5]:
psi0 = qt.tensor(qt.basis(2,0),qt.basis(2,1))
psi0*psi0.dag()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

In [199]:
I = qt.operators.identity(2)
Id = qt.tensor([I]*N)


In [246]:
coherent = -1j*(qt.tensor(Id,hamiltonian)-qt.tensor(hamiltonian.trans(),Id))
coherent

Quantum object: dims = [[2, 2, 2, 2], [2, 2, 2, 2]], shape = (16, 16), type = oper, isherm = False
Qobj data =
[[0.+0.j   0.-0.5j  0.-0.5j  0.+0.j   0.+0.5j  0.+0.j   0.+0.j   0.+0.j
  0.+0.5j  0.+0.j   0.+0.j   0.+0.j   0.+0.j   0.+0.j   0.+0.j   0.+0.j  ]
 [0.-0.5j  0.-0.25j 0.+0.j   0.-0.5j  0.+0.j   0.+0.5j  0.+0.j   0.+0.j
  0.+0.j   0.+0.5j  0.+0.j   0.+0.j   0.+0.j   0.+0.j   0.+0.j   0.+0.j  ]
 [0.-0.5j  0.+0.j   0.-0.25j 0.-0.5j  0.+0.j   0.+0.j   0.+0.5j  0.+0.j
  0.+0.j   0.+0.j   0.+0.5j  0.+0.j   0.+0.j   0.+0.j   0.+0.j   0.+0.j  ]
 [0.+0.j   0.-0.5j  0.-0.5j  0.-2.j   0.+0.j   0.+0.j   0.+0.j   0.+0.5j
  0.+0.j   0.+0.j   0.+0.j   0.+0.5j  0.+0.j   0.+0.j   0.+0.j   0.+0.j  ]
 [0.+0.5j  0.+0.j   0.+0.j   0.+0.j   0.+0.25j 0.-0.5j  0.-0.5j  0.+0.j
  0.+0.j   0.+0.j   0.+0.j   0.+0.j   0.+0.5j  0.+0.j   0.+0.j   0.+0.j  ]
 [0.+0.j   0.+0.5j  0.+0.j   0.+0.j   0.-0.5j  0.+0.j   0.+0.j   0.-0.5j
  0.+0.j   0.+0.j   0.+0.j   0.+0.j   0.+0.j   0.+0.5j  0.+0.j   0.+0.j  ]
 [0.+

In [247]:
sm = qt.operators.sigmam()

In [248]:
#sm = qt.operators.sigmam()
#sm = sx -1j*sy
kappa = 1
c_ops = [qt.tensor([sm if i==j  else I for j in range(N)]) for i in range(N)]
c_ops = [np.sqrt(kappa) * c_op for c_op in c_ops]
c_tops = [c.dag() for c in c_ops]

In [249]:
dissipative_part = sum([qt.tensor(Ad.trans(),A)-0.5*qt.tensor(Id,Ad*A)-0.5*qt.tensor((Ad*A).trans(),Id) for Ad,A in zip(c_tops,c_ops)])
dissipative_part

Quantum object: dims = [[2, 2, 2, 2], [2, 2, 2, 2]], shape = (16, 16), type = oper, isherm = False
Qobj data =
[[-2.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.  -1.5  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.  -1.5  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.  -1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.  -1.5  0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 1.   0.   0.   0.   0.  -1.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.  -1.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   1.   0.   0.   0.   0.  -0.5  0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.  -1.5  0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.  -1.   0.   0.   0.   0.
   0.   0. ]
 [ 1.   0.   0.   0.   0.   0.   0.   0.   0.   0

In [250]:
Linb = coherent + dissipative_part
Linb

Quantum object: dims = [[2, 2, 2, 2], [2, 2, 2, 2]], shape = (16, 16), type = oper, isherm = False
Qobj data =
[[-2. +0.j    0. -0.5j   0. -0.5j   0. +0.j    0. +0.5j   0. +0.j
   0. +0.j    0. +0.j    0. +0.5j   0. +0.j    0. +0.j    0. +0.j
   0. +0.j    0. +0.j    0. +0.j    0. +0.j  ]
 [ 0. -0.5j  -1.5-0.25j  0. +0.j    0. -0.5j   0. +0.j    0. +0.5j
   0. +0.j    0. +0.j    0. +0.j    0. +0.5j   0. +0.j    0. +0.j
   0. +0.j    0. +0.j    0. +0.j    0. +0.j  ]
 [ 0. -0.5j   0. +0.j   -1.5-0.25j  0. -0.5j   0. +0.j    0. +0.j
   0. +0.5j   0. +0.j    0. +0.j    0. +0.j    0. +0.5j   0. +0.j
   0. +0.j    0. +0.j    0. +0.j    0. +0.j  ]
 [ 0. +0.j    0. -0.5j   0. -0.5j  -1. -2.j    0. +0.j    0. +0.j
   0. +0.j    0. +0.5j   0. +0.j    0. +0.j    0. +0.j    0. +0.5j
   0. +0.j    0. +0.j    0. +0.j    0. +0.j  ]
 [ 0. +0.5j   0. +0.j    0. +0.j    0. +0.j   -1.5+0.25j  0. -0.5j
   0. -0.5j   0. +0.j    0. +0.j    0. +0.j    0. +0.j    0. +0.j
   0. +0.5j   0. +0.j    0. +0.j    0.

In [251]:
L = qt.liouvillian(hamiltonian, c_ops)
L

Quantum object: dims = [[[2, 2], [2, 2]], [[2, 2], [2, 2]]], shape = (16, 16), type = super, isherm = False
Qobj data =
[[-2. +0.j    0. -0.5j   0. -0.5j   0. +0.j    0. +0.5j   0. +0.j
   0. +0.j    0. +0.j    0. +0.5j   0. +0.j    0. +0.j    0. +0.j
   0. +0.j    0. +0.j    0. +0.j    0. +0.j  ]
 [ 0. -0.5j  -1.5-0.25j  0. +0.j    0. -0.5j   0. +0.j    0. +0.5j
   0. +0.j    0. +0.j    0. +0.j    0. +0.5j   0. +0.j    0. +0.j
   0. +0.j    0. +0.j    0. +0.j    0. +0.j  ]
 [ 0. -0.5j   0. +0.j   -1.5-0.25j  0. -0.5j   0. +0.j    0. +0.j
   0. +0.5j   0. +0.j    0. +0.j    0. +0.j    0. +0.5j   0. +0.j
   0. +0.j    0. +0.j    0. +0.j    0. +0.j  ]
 [ 0. +0.j    0. -0.5j   0. -0.5j  -1. -2.j    0. +0.j    0. +0.j
   0. +0.j    0. +0.5j   0. +0.j    0. +0.j    0. +0.j    0. +0.5j
   0. +0.j    0. +0.j    0. +0.j    0. +0.j  ]
 [ 0. +0.5j   0. +0.j    0. +0.j    0. +0.j   -1.5+0.25j  0. -0.5j
   0. -0.5j   0. +0.j    0. +0.j    0. +0.j    0. +0.j    0. +0.j
   0. +0.5j   0. +0.j    0. +

## Diagonalization

In [253]:
import scipy as sp

In [254]:
Ldag = Linb.dag().full()
Ldag

array([[-2. +0.j  ,  0. +0.5j ,  0. +0.5j ,  0. +0.j  ,  0. -0.5j ,
         1. +0.j  ,  0. +0.j  ,  0. +0.j  ,  0. -0.5j ,  0. +0.j  ,
         1. +0.j  ,  0. +0.j  ,  0. +0.j  ,  0. +0.j  ,  0. +0.j  ,
         0. +0.j  ],
       [ 0. +0.5j , -1.5+0.25j,  0. +0.j  ,  0. +0.5j ,  0. +0.j  ,
         0. -0.5j ,  0. +0.j  ,  0. +0.j  ,  0. +0.j  ,  0. -0.5j ,
         0. +0.j  ,  1. +0.j  ,  0. +0.j  ,  0. +0.j  ,  0. +0.j  ,
         0. +0.j  ],
       [ 0. +0.5j ,  0. +0.j  , -1.5+0.25j,  0. +0.5j ,  0. +0.j  ,
         0. +0.j  ,  0. -0.5j ,  1. +0.j  ,  0. +0.j  ,  0. +0.j  ,
         0. -0.5j ,  0. +0.j  ,  0. +0.j  ,  0. +0.j  ,  0. +0.j  ,
         0. +0.j  ],
       [ 0. +0.j  ,  0. +0.5j ,  0. +0.5j , -1. +2.j  ,  0. +0.j  ,
         0. +0.j  ,  0. +0.j  ,  0. -0.5j ,  0. +0.j  ,  0. +0.j  ,
         0. +0.j  ,  0. -0.5j ,  0. +0.j  ,  0. +0.j  ,  0. +0.j  ,
         0. +0.j  ],
       [ 0. -0.5j ,  0. +0.j  ,  0. +0.j  ,  0. +0.j  , -1.5-0.25j,
         0. +0.5j ,  0. +0.5j , 

In [255]:
val,mat = np.linalg.eig(Ldag)

In [256]:
val

array([-1.04150755e+00+2.86857516e+00j, -1.04150755e+00-2.86857516e+00j,
       -6.58131392e-01+1.66140372e+00j, -1.57399404e+00+1.32326328e+00j,
       -6.58131392e-01-1.66140372e+00j, -1.57399404e+00-1.32326328e+00j,
       -5.25510634e-17+1.90565975e-16j, -7.73323335e-01-8.63534685e-16j,
       -1.50049854e+00+5.60946366e-16j, -1.17891216e+00-1.50983118e-17j,
       -5.67634222e-01+2.00475397e+00j, -5.67634222e-01-2.00475397e+00j,
       -1.24888595e+00+9.16394509e-01j, -1.24888595e+00-9.16394509e-01j,
       -1.18347982e+00-2.20752899e-01j, -1.18347982e+00+2.20752899e-01j])

In [244]:
mat

array([[ 3.87306553e-01-0.j,  1.61905034e-02-0.j,  7.41620338e-01+0.j,
        -2.53585809e-02-0.j,  6.46461316e-01+0.j, -1.48079530e-01+0.j,
         5.05594367e-01+0.j,  5.00000000e-01+0.j,  5.73832142e-01+0.j,
         2.54015031e-01+0.j,  1.67184640e-17-0.j,  1.75646391e-15+0.j,
        -2.79874273e-16+0.j, -6.33754473e-17-0.j,  1.82598139e-14-0.j,
         6.94562350e-15-0.j],
       [-1.35976026e-01-0.j,  9.34094653e-02-0.j, -3.30255413e-01-0.j,
         3.22704762e-01-0.j, -2.77616775e-01-0.j,  5.05593066e-01+0.j,
        -2.17306997e-01+0.j,  1.09912164e-15-0.j, -2.16073289e-01+0.j,
        -3.41509809e-01+0.j,  2.50122534e-01-0.j, -1.24767333e-01+0.j,
        -4.43009091e-01+0.j, -1.12817761e-01-0.j,  1.31308566e-02-0.j,
         3.31931454e-01-0.j],
       [-1.35976026e-01-0.j,  9.34094653e-02-0.j, -3.30255413e-01-0.j,
         3.22704762e-01-0.j, -2.77616775e-01-0.j,  5.05593066e-01+0.j,
        -2.17306997e-01+0.j,  8.24341227e-16-0.j, -2.16073289e-01+0.j,
        -3.415098

In [245]:
# checking whether l1 is identity

l1indx = np.argsort(abs(val.real))[0]
l1 = mat.copy()[l1indx]
l1 # not working

array([ 3.58669890e-02-0.j,  5.67422867e-01+0.j,  3.02682425e-02-0.j,
        3.65366988e-01-0.j, -2.96083919e-02-0.j, -9.01909389e-03+0.j,
       -7.09526106e-02+0.j,  1.69095636e-16-0.j, -1.28595601e-01+0.j,
       -1.62685690e-01+0.j, -6.26177699e-01-0.j, -4.43405793e-02+0.j,
       -8.27765731e-02+0.j, -8.54645961e-02-0.j,  1.75382482e-01-0.j,
        6.44362620e-02-0.j])

In [ ]:
l2indx=np.argsort(abs(val.real))[1]
l2 = mat.copy()[l2indx]
#l2[l2 < 1e-10] = 0

In [229]:
l2

array([-1.28357581e-01-0.j,  8.59208441e-02-0.j, -1.04137011e-01-0.j,
        7.31303551e-03-0.j, -1.09723565e-02-0.j,  3.08132986e-02+0.j,
        3.42871602e-01+0.j,  2.11369545e-16-0.j,  1.97721961e-01+0.j,
        3.02841124e-01+0.j,  8.27663715e-02-0.j, -3.06255237e-01+0.j,
       -3.77398547e-01+0.j, -2.99256586e-01-0.j,  1.50229192e-01-0.j,
       -2.78175055e-01-0.j])

In [230]:
#l2 = np.real(l2)+1j*np.real_if_close(np.imag(l2), tol=1e-10)
l2 = np.real_if_close(l2, tol=1e-10)

In [231]:
l2[abs(l2) < 1e-10] = 0
l2

array([-0.12835758,  0.08592084, -0.10413701,  0.00731304, -0.01097236,
        0.0308133 ,  0.3428716 ,  0.        ,  0.19772196,  0.30284112,
        0.08276637, -0.30625524, -0.37739855, -0.29925659,  0.15022919,
       -0.27817505])

In [232]:
l2 = l2.reshape(2**N,2**N)
l2 

array([[-0.12835758,  0.08592084, -0.10413701,  0.00731304],
       [-0.01097236,  0.0308133 ,  0.3428716 ,  0.        ],
       [ 0.19772196,  0.30284112,  0.08276637, -0.30625524],
       [-0.37739855, -0.29925659,  0.15022919, -0.27817505]])

In [233]:
np.matrix(l2).H

matrix([[-0.12835758, -0.01097236,  0.19772196, -0.37739855],
        [ 0.08592084,  0.0308133 ,  0.30284112, -0.29925659],
        [-0.10413701,  0.3428716 ,  0.08276637,  0.15022919],
        [ 0.00731304,  0.        , -0.30625524, -0.27817505]])

### Eigenvalues of $l_{2}$

In [107]:
alpha,_ = np.linalg.eig(l2)

In [ ]:
alpha # eigen values should be real

array([ 0.38366297+0.j        , -0.0990989 +0.17932299j,
       -0.0990989 -0.17932299j,  0.        +0.j        ])